In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
pd.set_option('max_columns', 100)
pd.set_option('max_colwidth', 100)
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
merchant = pd.read_csv('data/merchants.csv')

# 数据观察

In [14]:
merchant.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,1.666667,3,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,0.500000,3,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [5]:
states = []
for col in merchant.columns:
    states.append((col, merchant[col].nunique(), merchant[col].isnull().sum()/merchant.shape[0], merchant[col].value_counts(normalize=True, dropna=False).values[0], merchant[col].dtype))
state_df = pd.DataFrame(states, columns=['Feature', 'Unique values', 'Missing values', 'Percentage of biggest category', 'type'])
state_df.sort_values('Percentage of biggest category', ascending=False)

,Feature,Unique values,Missing values,Percentage of biggest category,type
11,active_months_lag3,3,0.000000,0.995716,int64
6,category_1,2,0.000000,0.978969,object
14,active_months_lag6,6,0.000000,0.977855,int64
17,active_months_lag12,12,0.000000,0.911576,int64
5,numerical_2,947,0.000000,0.743104,float64
18,category_4,2,0.000000,0.712874,object
4,numerical_1,954,0.000000,0.683570,float64
7,most_recent_sales_range,5,0.000000,0.529149,object
8,most_recent_purchases_range,5,0.000000,0.523786,object
21,category_2,5,0.035516,0.480699,float64


# numerical特征处理

In [3]:
sub_num2 = ['numerical_1', 'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6', 'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12']

## mf合并与merchant_id相关的特征

In [4]:
mf = merchant[['merchant_id']]

In [5]:
for i in sub_num2:
    t1 = merchant[['merchant_id', i]].groupby('merchant_id').mean().reset_index()
    t1 = t1.rename(columns={i: 'merchant_'+i})
    mf = pd.merge(mf, t1, on=['merchant_id'], how='left')

In [6]:
mgf = merchant[['merchant_id', 'merchant_group_id']]

In [7]:
for i in sub_num2:
    t2 = merchant[['merchant_group_id', i]].groupby('merchant_group_id').mean().reset_index()
    t2 = t2.rename(columns={i: 'mgroup_'+i})
    mgf = pd.merge(mgf, t2, on=['merchant_group_id'], how='left')

In [8]:
mf = pd.merge(mf, mgf, on=['merchant_id'], how='left')

## mcf合并与merchant_category_id相关的特征

In [9]:
mcf = merchant[['merchant_category_id']]

In [10]:
for i in sub_num2:
    t3 = merchant[['merchant_category_id', i]].groupby('merchant_category_id').mean().reset_index()
    t3 = t3.rename(columns={i: 'mcategory_'+i})
    mcf = pd.merge(mcf, t3, on=['merchant_category_id'], how='left')

## sf合并与subsector_id相关的特征

In [11]:
sf = merchant[['subsector_id']]

In [12]:
for i in sub_num2:
    t4 = merchant[['subsector_id', i]].groupby('subsector_id').mean().reset_index()
    t4 = t4.rename(columns={i: 'subsector_'+i})
    sf = pd.merge(sf, t4, on=['subsector_id'], how='left')

# category特征处理

In [13]:
sub_cate2 = ['category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4', 'category_2']